# Population Projections

## Resources:
[pandas Melt](https://pandas.pydata.org/docs/reference/api/pandas.melt.html)
<br>[pandas Query](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

dfUrl = 'https://www2.census.gov/programs-surveys/popproj/datasets/2017/2017-popproj/np2017_d1_mid.csv'

df = pd.read_csv(dfUrl)
df.info()

In [ ]:
print(f'Rows: {df.shape[0]:,}')
print(f'Columns: {df.shape[1]:,}')

In [ ]:
df.columns

In [ ]:
df.head()

## File layout
- SEX: sex, see key  
- ORIGIN: Hispanic origin, see key  
- RACE: see key  
- YEAR: year of population projection  
- TOTAL_POP: total population, all ages combined  
- (POP_0, POP_1, ...POP_99, POP_100): population for given age  

The key for ORIGIN is as follows:
- 0 = Total  
- 1 = Not Hispanic  
- 2 = Hispanic  

The key for RACE is as follows:
- 0 = All races (codes 1 through 6)  
- 1 = White alone  
- 2 = Black alone  
- 3 = AIAN alone  
- 4 = Asian alone  
- 5 = NHPI alone  
- 6 = Two or More Races  
- 7 = White alone or in combination  
- 8 = Black alone or in combination  
- 9 = AIAN alone or in combination  
- 10 = Asian alone or in combination  
- 11 = NHPI alone or in combination

The key for SEX is as follows:
- 0 = Both sexes  
- 1 = Male  
- 2 = Female  
      
NOTE: Hispanic origin is considered an ethnicity, not a race. Hispanics may be of any race.

## Original question:
We need to understand what the 85+ population looks like over time and understand trends in its annual growth rate. Probably don't need to worry about ORIGIN, RACE, or SEX so we can filter down to the aggregate values.

## Things to consider
- This dataset is very wide, likely want to consider converting to a long format  
- There are aggregations included with the raw data
- Haven't determined if there are nulls/missing data yet   

### Let's get checking for null values out of the way!

In [ ]:
df.isna().sum().value_counts(dropna=False)

In [ ]:
df.dropna().shape

__No missing values in any of the columns!__

### But we still have a very wide dataset, so we should convert it to long dataset via "melt".
And we'll want to filter down to the overall population by year.

In [ ]:
# df.query('ORIGIN==0 and RACE==0 and SEX==0') \
# .drop(columns=['ORIGIN','RACE','SEX','TOTAL_POP']) \
# .melt(id_vars='YEAR', var_name='POP_AGE', value_name='POP') \
# .query('YEAR==2016')


In [ ]:
# (
#     df.query("ORIGIN==0 and RACE==0 and SEX==0")
#     .drop(columns=['ORIGIN','RACE','SEX','TOTAL_POP'])
#     .melt(id_vars='YEAR', var_name='POP_AGE', value_name='POP')
# )

In [ ]:
dfLong = (
    df.query("ORIGIN==0 and RACE==0 and SEX==0")
    .drop(columns=['ORIGIN','RACE','SEX','TOTAL_POP'])
    .melt(id_vars='YEAR', var_name='POP_AGE', value_name='POP')
)

dfLong.head()

### Extract the age

In [ ]:

# dfLong['AGE'] = dfLong['POP_AGE'].apply(lambda x: int(x.split('_')[1]))
dfLong['AGE'] = [ int(x.split('_')[1]) for x in dfLong['POP_AGE'] ]
dfLong = dfLong.drop(columns=['POP_AGE'])
dfLong.head()

In [ ]:
# dfLong.query('YEAR==2016').head()

### Let's look at the distribution for 2021

In [ ]:
dfLong.query("YEAR==2021")['POP'].sum()

In [ ]:
total2021 = dfLong.query("YEAR==2021")["POP"].sum()

In [ ]:
(
    dfLong.query('YEAR==2021')
    .plot(x='AGE', y='POP', kind='line', legend=False)
)

plt.title(f'US age distribution for 2021 (total={total2021:,})', loc='left')
plt.xlabel('Age')
plt.ylabel('# of people')
plt.show()

### Make sure that total squares: https://www.census.gov
You want to avoid showing data that is levels of magnitude out of expected bounds.

### How will this shape compare against 2050?

In [ ]:
# (
#     dfLong.query('YEAR==2021 or YEAR==2050')
#     .pivot_table(index='AGE', columns='YEAR', values='POP')
# )

In [ ]:
yearCompare = (
    dfLong.query('YEAR==2021 or YEAR==2050')
    .pivot_table(index='AGE', columns='YEAR', values='POP')
)

yearCompare.head()

In [ ]:
yearCompare.plot(kind='line')
plt.title(f'US age distribution for 2021 vs. 2050', loc='left')
plt.xlabel('Age')
plt.ylabel('# of people')
plt.legend(title='')
plt.show()

__This is probably more meaningful on a percent basis since population will grow over time. We should probably account for population differences and measure this on a percent basis.__

In [ ]:
yearCompare.head()

In [ ]:
yearCompareSum = yearCompare.sum(axis=0)
yearCompareSum

[DataFrame.div](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.div.html)

In [ ]:
(
    yearCompare.div(yearCompareSum, axis=1)
    .plot(kind='line')
)

plt.title(f'US age distribution for 2021 vs. 2050', loc='left')
plt.xlabel('Age')
plt.ylabel('% of population')
plt.legend(title='')
plt.show()

#### How does this compare with the prior plot?
This shows that on average, the population is aging. The original plot seemed to show more younger individuals in the out year, but that's a product of a larger overall population. We should observe the growth rate for the older population increasing, as a way to check our math.

### Let's create our growth rate plot

In [ ]:
pop85 = (
    dfLong.query('AGE>=85')
    .groupby('YEAR')['POP']
    .sum()
)
pop85.head()

[DataFrame.shift](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shift.html)

In [ ]:
pop85.head().shift(1)

In [ ]:
pop85Growth = ( pop85 / pop85.shift(1) ) - 1
pop85Growth.head()

In [ ]:
pop85Growth.tail()

In [ ]:
pop85Growth.plot(kind='line')

plt.title('Growth rate of 85+ population', loc='left')
plt.show()

__Growth rates help show the growth, but having a baseline, maybe the under 84 age group would be helpful to see the contrast.__

In [ ]:
popU84 = (
    dfLong.query('AGE<85')
    .groupby('YEAR')['POP'].sum()
)

popU84Growth = (popU84 / popU84.shift(1)) - 1

pop85Growth.plot(kind='line')
popU84Growth.plot(kind='line')
plt.title('Growth rate 85+ population versus under 85 population', loc='left')
plt.legend(['85 and over','Under 85'], title='Age Group')
plt.show()

## We can also look at different parts of the distribution over time?
We can cut the data into bins, if you aren't familiar, you can look at the documentation:

```python
help(pd.cut)
```

[DataFrame.cut](https://pandas.pydata.org/docs/reference/api/pandas.cut.html)

In [ ]:
dfLong.describe()

In [ ]:
dfLong['AGE_BINS'] = pd.cut(dfLong['AGE'], 6)
dfLong

In [ ]:
dfLong['AGE_BINS'].value_counts()

In [ ]:
( dfLong
    .pivot_table(index='YEAR', columns='AGE_BINS', values='POP', aggfunc='sum')
).head()

In [ ]:
(
    dfLong
    .pivot_table(index='YEAR', columns='AGE_BINS', values='POP', aggfunc='sum')
    .plot(kind='line', subplots=True, layout=(2,3), figsize=(8,8))
)
plt.show()

This is interesting. Beginning in the mid-2020s into the early-2030s, some groups are actually experiencing declines in their population. It looks like there will also be unfettered growth in the youngest age groups.

__Let's peel the onion more. Let's look at which races may be driving some of these trends.__
> Remember "0 = All races (codes 1 through 6)  "

In [ ]:
raceFilter = [1,2,3,4,5,6]

raceMap = {1:'White alone',
           2:'Black alone',
           3:'AIAN alone', 
           4:'Asian alone', 
           5:'NHPI alone', 
           6:'Two or More Races'
          }

In [ ]:
dfLongRace = (
    df.query("ORIGIN==0 and RACE==@raceFilter and SEX==0")
    .drop(columns=['ORIGIN','SEX','TOTAL_POP'])
    .melt(id_vars=['YEAR','RACE'], var_name='POP_AGE', value_name='POP')
)

dfLongRace.head()

In [ ]:
dfLongRace['RACE_DESC'] = dfLongRace['RACE'].map(raceMap)
dfLongRace['AGE'] = dfLongRace['POP_AGE'].apply(lambda x: int(x.split('_')[1]))
dfLongRace = dfLongRace.drop(columns=['POP_AGE', 'RACE'])

dfLongRace.head()

In [ ]:
dfRacePivot = dfLongRace.pivot_table(index='YEAR', columns='RACE_DESC', values='POP', aggfunc='sum')
dfRacePivot.head()

In [ ]:
dfRacePivot.iloc[0,:] # 2016

In [ ]:

(dfRacePivot/dfRacePivot.iloc[0,:]).plot(kind='line')
plt.title('Population growth by race, indexed to 100 in 2016')
plt.legend(title='')
plt.xlabel('')
plt.show()

__Looks like diversity is expected to increase in the United States, but what about how that translates to the 85+ population?__

In [ ]:
dfLongRace.head()

In [ ]:
dfRacePivot = (
    dfLongRace.query('AGE >= 85')
    .pivot_table(index='YEAR', columns='RACE_DESC', values='POP', aggfunc='sum')
)

dfRacePivot.head()

In [ ]:
(dfRacePivot/dfRacePivot.shift(1) - 1).plot(figsize=(10,6))
plt.legend(title='')
plt.xlabel('')
plt.ylabel('')
plt.title('Projected 85+ Population Growth Rates by Race', loc='left')
plt.show()